In [42]:
import os
import glob
import argparse

import cv2
import numpy as np
from keras.models import Model
from keras.layers import Dense, Activation, MaxPool2D, Conv2D, Flatten, Dropout, Input, BatchNormalization, Add
from keras.optimizers import Adam
from keras.utils import multi_gpu_model, plot_model

# Keras 內建模型
# https://keras.io/applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.applications.mobilenetv2 import MobileNetV2

In [55]:
# 資料參數
input_height = 48
input_width = 48
input_channel = 3
input_shape = (input_height, input_width, input_channel)
n_classes = 4                               # 總共有 left, right, stop, other 四種
plot_model_file = 'model_architecture.png'  # 輸出模型架構圖，設定 None 表示不繪圖
data_dir = os.path.expanduser('~/dataset')  # 資料集路徑，這預設爲家目錄下的 dataset 目錄
epochs = 20                                 # 訓練資料集的回合數
batch_size = 64
num_gpu = 1
model_type = 'custom'   # 'VGG16', 'VGG19', 'ResNet50', 'DenseNet121', 'MobileNetV2', 'custom' 之一

In [53]:
# 定義自定模型架構，供下段選用
def custom_model(input_shape, n_classes):
    def conv_block(x, filters):
        x = BatchNormalization() (x)
        x = Conv2D(filters, (3, 3), activation='relu', padding='same') (x)

        x = BatchNormalization() (x)
        shortcut = x
        x = Conv2D(filters, (3, 3), activation='relu', padding='same') (x)
        x = Add() ([x, shortcut])
        x = MaxPool2D((2, 2), strides=(2, 2)) (x)

        return x

    input_tensor = Input(shape=input_shape)

    x = conv_block(input_tensor, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    x = conv_block(x, 512)

    x = Flatten() (x)
    x = BatchNormalization() (x)
    x = Dense(512, activation='relu') (x)
    x = Dense(512, activation='relu') (x)

    output_layer = Dense(n_classes, activation='softmax') (x)

    inputs = [input_tensor]
    model = Model(inputs, output_layer)

    return model

In [45]:
# 建立模型
if model_type == 'VGG16':
    input_tensor = Input(shape=input_shape)
    model = VGG16(
        input_shape=input_shape,
        classes=n_classes,
        weights=None,
        input_tensor=input_tensor,
    )
elif model_type == 'VGG19':
    input_tensor = Input(shape=input_shape)
    model = VGG19(
        input_shape=input_shape,
        classes=n_classes,
        weights=None,
        input_tensor=input_tensor,
    )
elif model_type == 'ResNet50':
    input_tensor = Input(shape=input_shape)
    model = ResNet50(
        input_shape=input_shape,
        classes=n_classes,
        weights=None,
        input_tensor=input_tensor,
    )
elif model_type == 'DenseNet121':
    input_tensor = Input(shape=input_shape)
    model = DenseNet121(
        input_shape=input_shape,
        classes=n_classes,
        weights=None,
        input_tensor=input_tensor,
    )
elif model_type == 'MobileNetV2':
    input_tensor = Input(shape=input_shape)
    model = MobileNetV2(
        input_shape=input_shape,
        classes=n_classes,
        weights=None,
        input_tensor=input_tensor,
    )
elif model_type == 'custom':
    model = custom_model(input_shape, n_classes)

if num_gpu > 1:
    model = multi_gpu_model(model, gpus=num_gpu)

if plot_model_file is not None:
    plot_model(model, to_file=plot_model_file)

In [46]:
# 初始化優化器，並指定損失函數
adam = Adam()
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['acc'],
)

In [47]:
# 表列資料集下所有圖片路徑
match_left = os.path.join(data_dir, 'left', '*.jpg')
paths_left = glob.glob(match_left)

match_right = os.path.join(data_dir, 'right', '*.jpg')
paths_right = glob.glob(match_right)

match_stop = os.path.join(data_dir, 'stop', '*.jpg')
paths_stop = glob.glob(match_stop)

match_other = os.path.join(data_dir, 'other', '*.jpg')
paths_other = glob.glob(match_other)

match_test = os.path.join(data_dir, 'test', '*.jpg')
paths_test = glob.glob(match_test)

n_train = len(paths_left) + len(paths_right) + len(paths_stop) + len(paths_other)
n_test = len(paths_test)

In [48]:
## 建立訓練及測試集

# 初始化資料集矩陣
trainset = np.zeros(
    shape=(n_train, input_height, input_width, input_channel),
    dtype='float32',
)
label = np.zeros(
    shape=(n_train, n_classes),
    dtype='float32',
)
testset = np.zeros(
    shape=(n_test, input_height, input_width, input_channel),
    dtype='float32',
)

# 讀取圖片到資料集
paths_train = paths_left + paths_right + paths_stop + paths_other

for ind, path in enumerate(paths_train):
    image = cv2.imread(path)
    resized_image = cv2.resize(image, (input_width, input_height))
    trainset[ind] = resized_image

for ind, path in enumerate(paths_test):
    image = cv2.imread(path)
    resized_image = cv2.resize(image, (input_width, input_height))
    testset[ind] = resized_image

# 由於原圖數值是 0 ~ 255，我們正規化數值到 0～1 之間
trainset = trainset / 255.0
testset = testset / 255.0
    
# 設定訓練集的標記
n_left = len(paths_left)
n_right = len(paths_right)
n_stop = len(paths_stop)
n_other = len(paths_other)

begin_ind = 0
end_ind = n_left
label[begin_ind:end_ind, 0] = 1.0

begin_ind = n_left
end_ind = n_left + n_right
label[begin_ind:end_ind, 1] = 1.0

begin_ind = n_left + n_right
end_ind = n_left + n_right + n_stop
label[begin_ind:end_ind, 2] = 1.0

begin_ind = n_left + n_right + n_stop
end_ind = n_left + n_right + n_stop + n_other
label[begin_ind:end_ind, 3] = 1.0

In [49]:
# 訓練模型
model.fit(
    trainset,
    label,
    epochs=epochs,
    validation_split=0.2,
    # batch_size=64,
)

Train on 1732 samples, validate on 433 samples
Epoch 1/20
1732/1732 [==============================] - 8s 5ms/step - loss: 0.3281 - acc: 0.8793 - val_loss: 7.6301 - val_acc: 0.2171
Epoch 2/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.0892 - acc: 0.9711 - val_loss: 1.3073 - val_acc: 0.8037
Epoch 3/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.0414 - acc: 0.9856 - val_loss: 0.7615 - val_acc: 0.8707
Epoch 4/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.1132 - acc: 0.9654 - val_loss: 0.8082 - val_acc: 0.8268
Epoch 5/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.0885 - acc: 0.9769 - val_loss: 0.9599 - val_acc: 0.8661
Epoch 6/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.1297 - acc: 0.9625 - val_loss: 8.9052 - val_acc: 0.1501
Epoch 7/20
1732/1732 [==============================] - 3s 2ms/step - loss: 0.1906 - acc: 0.9457 - val_loss: 4.7464 - val_acc: 0.2194
Epoch 8/20
1732

In [51]:
# 執行預測
if testset.shape[0] != 0:
    result_onehot = model.predict(testset)
    result_sparse = np.argmax(result_onehot, axis=1)
else:
    result_sparse = list()

# 印出預測結果
print('檔名\t預測類別')

for path, label_id in zip(paths_test, result_sparse):
    filename = os.path.basename(path)

    if label_id == 0:
        label_name = 'left'
    elif label_id == 1:
        label_name = 'right'
    elif label_id == 2:
        label_name = 'stop'
    elif label_id == 3:
        label_name = 'other'

    print('%s\t%s' % (filename, label_name))

檔名	預測類別
006b3da16935de94b4703c8f9602e9c4d43218ce.jpg	left
0156f8e07265127674f9a182f39e67ebf5d728ed.jpg	left
026888e0e32be089d4f5dda0c90d72498b460202.jpg	left
02fc6c1f4c5783331b465abf688659665615c571.jpg	left
04e8a90ad5bd18d93762f8e057fd3c298d2d8d28.jpg	left
05fc0db22502e8ac033c95f0df566f0bfa2faff2.jpg	left
0889afaf993245d3e8cecaf32094f41c1b328426.jpg	left
09a4f9623b862884f09a36ae8a34fcbacd6dee50.jpg	left
0bc48351f767f35a0a4a9509ca3b44531c354e0c.jpg	left
0c1a8c69ea5efedeadd031cd4e3430d44d1fb1c7.jpg	left
0c4591b88f0ddbac0e0a99e9d4bfdd3e926eaf50.jpg	left
0d35694fed48299aeb2ed7432af9d24d0aba3f38.jpg	left
0d72cec7d1ddbcb47b5225dadd984db18a274cae.jpg	left
1001cf7e06def1ad9339562f6294e518f14a62d4.jpg	left
11540b4992c587dbe238b1625ca6493b064f3740.jpg	left
11b2515f6f1f20211452fcad4937da69c78436d8.jpg	left
13c8e5b71ff716dfca21015f5be1fd24364ed486.jpg	left
13da0517d13015f6f16595a52ba610fbb5d68ad4.jpg	left
1516fc6495d463c22daec800d5c37a7d36d26af0.jpg	left
158e7a5446d5f509dcdf00ab504d2f8cf590f61a.j